# Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from helpers import parse_variables, get_risk_level, hi_gauss_blob_risk_fun, blob_risk_fun, NW_risk_fun, square_risk_fun, map_to_color
from matplotlib.colors import LinearSegmentedColormap
import importlib.util
from k_means_constrained import KMeansConstrained


import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from helpers import parse_variables, get_risk_level, map_to_color, simulate_quant_trait
from models import ols_regression, manhattan_linear, gc
from deep_learning_models import abyss, deep_abyss

from scipy.stats import t
from scipy import stats
from scipy.stats import entropy

from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras import Input, Model, layers, regularizers
from tensorflow.keras.layers import Input, Dense

2024-08-28 13:05:33.197489: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 13:05:33.204047: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 13:05:33.219946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 13:05:33.251626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 13:05:33.251713: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 13:05:33.274941: I tensorflow/core/platform/cpu_feature_guard.cc:

In [6]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
k = int(dict['k'])
M = float(dict['M'])
HWE = int(dict['HWE'])

nr_humans = int(dict['nr_humans'])
nr_snps = int(dict['nr_snps'])
bottleneck_nr = int(dict['bottleneck_nr'])

# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k

In [7]:
very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

In [8]:
very_rare = very_rare.rename(columns=lambda x: 'VR' + x)/2
rare = rare.rename(columns=lambda x: 'R' + x)/2
common = common.rename(columns=lambda x: 'C' + x)/2
complete = pd.concat([common, rare, very_rare], axis=1)
complete = ((complete*2)-1)

In [11]:
bottle_file

'abyss_bottleneck_64_77.663seconds.pkl'

In [12]:
path_bottle = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/abyss_bottleneck"
bottle_file = [f for f in os.listdir(path_bottle) if int(f.split("_")[2]) ==  bottleneck_nr][0]
elapsed_time_bottleneck = float(bottle_file.split('_')[3].split('seconds')[0])
bottle = pd.read_pickle(f"{path_bottle}/{bottle_file}")

In [18]:
bottle

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim56,dim57,dim58,dim59,dim60,dim61,dim62,dim63,dim64,cluster
0,0.247320,0.558759,0.160977,0.071021,0.275661,0.586227,0.132713,0.050002,0.388084,0.185918,...,0.732892,0.081761,0.121670,0.042011,0.380503,0.055630,0.244014,0.363884,0.061893,0
1,0.209328,0.565148,0.142860,0.137045,0.270371,0.511784,0.089762,0.069028,0.421283,0.183521,...,0.691966,0.118586,0.081019,0.066986,0.361386,0.070954,0.285308,0.397945,0.085473,0
2,0.251229,0.488425,0.144627,0.170955,0.267871,0.484362,0.115141,0.075896,0.442830,0.175069,...,0.705041,0.165929,0.109381,0.025480,0.347549,0.061252,0.314206,0.434111,0.121496,0
3,0.213875,0.553388,0.099140,0.048397,0.300691,0.526767,0.070804,0.061034,0.473475,0.148031,...,0.829637,0.121872,0.061557,0.005857,0.425308,0.048374,0.268701,0.446401,0.069396,0
4,0.235143,0.513232,0.143646,0.123940,0.270662,0.521322,0.119588,0.082772,0.425415,0.178843,...,0.736520,0.134714,0.111598,0.026647,0.376465,0.068374,0.287013,0.417703,0.096154,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.280419,0.251336,0.474595,0.161040,0.481268,0.700591,0.164930,0.456915,0.405494,0.193323,...,0.159023,0.183242,0.117168,0.507177,0.427534,0.392159,0.696687,0.123606,0.104331,0
796,0.258084,0.273270,0.430764,0.172704,0.486466,0.661814,0.118411,0.469123,0.403519,0.188910,...,0.182314,0.204074,0.088481,0.497135,0.439936,0.379171,0.701129,0.115604,0.092330,0
797,0.323858,0.248029,0.432062,0.209108,0.498652,0.672147,0.142135,0.422136,0.396271,0.177328,...,0.151385,0.242962,0.100492,0.476889,0.394025,0.340653,0.700534,0.105706,0.120453,0
798,0.250809,0.316161,0.414623,0.137480,0.407878,0.637892,0.140013,0.420557,0.366142,0.204251,...,0.205633,0.223868,0.086058,0.463140,0.397854,0.369307,0.661276,0.142000,0.092324,0


66

# Run Abyss on LD block

In [22]:
def maf_prediction(bottle_in, geno_out, epoch, patience):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(bottle_in, geno_out, test_size=0.2, random_state=42)
    
    # Regularization parameter
    l2_regularizer = 0.001
    
    # Original autoencoder model with L2 regularization
    decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(int(nr_snps/2), activation='elu', input_shape=(bottle_in.shape[1],), kernel_regularizer=regularizers.l2(l2_regularizer)),  # First hidden layer with L2 regularization
        layers.BatchNormalization(),
        tf.keras.layers.Activation('elu'),
        tf.keras.layers.Dense(geno_out.shape[1], activation='linear', kernel_regularizer=regularizers.l2(l2_regularizer))  # Output layer
    ])
    
    # Compile the original model with L2 regularization
    decoder.compile(optimizer='adam',
                        loss='mean_squared_error',
                        metrics=['mean_absolute_error'])
    
    # Define Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    
    # Fit the original model with Early Stopping
    history = decoder.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    return decoder, history

In [ ]:
for pop in bottle['cluster'].unique():
    temp_bottle = bottle[bottle['cluster'] == pop]
    temp_bottle = temp_bottle.drop('cluster', axis=1)
    path_output = f"data/G{G}_L{L}_c{c}_k{k}_M{M}/genotype/LD_blocks_estimated_mafs/{pop}"
    os.system(f"rm -rf {path_output}")
    os.makedirs(path_output, exist_ok = True)
    path_one_hot_genotype = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks_one_hot/{pop}"
    path_lds = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks/{pop}"
    ld_files = os.listdir(path_lds)
    epoch = 200
    patience = 15
    p2s = []
    twopqs = []
    q2s = []

    for ld_file in ld_files:
        db_minor = pd.read_pickle(f"{path_one_hot_genotype}/{ld_file.split('.pkl')[0]}_db_minor.pkl")
        db_het = pd.read_pickle(f"{path_one_hot_genotype}/{ld_file.split('.pkl')[0]}_db_het.pkl")
        db_major = pd.read_pickle(f"{path_one_hot_genotype}/{ld_file.split('.pkl')[0]}_db_major.pkl")
        
        decoder, history = maf_prediction(temp_bottle, db_major, epoch, patience)
        p2 = decoder(tf.convert_to_tensor(temp_bottle, dtype=tf.float32))
        p2 = pd.DataFrame(data=p2, columns = db_major.columns)
    
        p2.index = db_major.index
        p2s.append(p2)
    
        decoder, history = maf_prediction(temp_bottle, db_het, epoch, patience)
        twopq = decoder(tf.convert_to_tensor(temp_bottle, dtype=tf.float32))
        twopq = pd.DataFrame(data=twopq, columns = db_het.columns)
        
        twopq.index = db_het.index
        twopqs.append(twopq)
    
        decoder, history = maf_prediction(temp_bottle, db_minor, epoch, patience)
        q2 = decoder(tf.convert_to_tensor(temp_bottle, dtype=tf.float32))
        q2 = pd.DataFrame(data=q2, columns = db_minor.columns)
        
        q2.index = db_minor.index
        q2s.append(q2)
        print("TOP")

    path_output_global = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/maf_pred/{pop}"
    p2 = pd.concat(p2s, axis=1)
    p2 = p2.sort_index()
    p2 = p2[list(complete.columns)]
    
    p2.to_pickle(f"{path_output_global}/esti_p2_via_esti_pop.pkl")

    q2 = pd.concat(q2s, axis=1)
    q2 = q2.sort_index()
    q2 = q2[list(complete.columns)]
    q2.to_pickle(f"{path_output_global}/esti_q2_via_esti_pop.pkl")


    twopq = pd.concat(twopqs, axis=1)
    twopq = twopq.sort_index()
    twopq = twopq[list(complete.columns)]
    twopq.to_pickle(f"{path_output_global}/esti_2pq_via_esti_pop.pkl")